# Bioimage Model Zoo Core  Example notebook

This notebook shows how to interact with the `bioimageio.core` programmatically to explore, load, use, and export content from the [BioImage Model Zoo](https://bioimage.io)."

## 0. Activate human readable output error messages and load dependencies

If the notebook is being run on Google Colab, install necessary dependencies

In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
     %pip install bioimageio.core torch onnxruntime

Enable pretty validation errors

In [ ]:
# enable pretty validation errors in ipynb
from bioimageio.spec.pretty_validation_errors import (
    enable_pretty_validation_errors_in_ipynb,
)
enable_pretty_validation_errors_in_ipynb()

Load general dependencies

In [ ]:
# Load general dependencies
from pathlib import Path
from ruyaml import YAML
from imageio.v2 import imread
from devtools import pprint
from bioimageio.spec.utils import download
from bioimageio.core import Tensor
from typing import Any, Dict, Union
from numpy.typing import NDArray
import matplotlib.pyplot as plt

# helper function for showing multiple images in napari
try:
    import napari
except ImportError:

    def show_images(images: Dict[str, Union[Tensor, NDArray[Any], Path]]):
        for name, im in images.items():
            if isinstance(im, Path):
                im = imageio.imread(im)
            elif isinstance(im, Tensor):
                im = im.data
            print(f"{name}: {im.shape}")

else:
    def show_images(images: Dict[str, Union[Tensor, NDArray[Any], Path]]):
        v = napari.Viewer()
        for name, im in images.items():
            if isinstance(im, Path):
                im = imageio.imread(im)
            elif isinstance(im, Tensor):
                im = im.data
            print(f"napari viewer: adding {name}")
            v.add_image(im, name=name)

## 1. Load a model

### 1.1 Inspect available models in the Bioimage Model Zoo

Go to https://bioimage.io to browser available models

### 1.2 Load model from the BioImage Model Zoo

Bioimage Model Zoo Models can be loaded via the model __nickname ID__, the __DOI__ or the __URL__ of the model's rdf.yaml file. 
To use a local model file, you can provide the path to the model __rdf.yaml__, or a .zip containing the rdf.yaml.

__WARNING__ Using a draft version of the model is not recommended, this version as not been reviewed by the BioImage Model Zoo team and may contain harmful code. Use at your own risk.

In [ ]:
BMZ_MODEL_ID = ""#"affable-shark"
BMZ_MODEL_DOI = "" #"10.5281/zenodo.6287342"
BMZ_MODEL_URL = "https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/draft/files/rdf.yaml"

load_description is a function of the `bioimageio.spec` package, but as it is a sub-package of `bioimageio.core` it can be called as `bioimageio.core.load_description`.
To learn more about the functionalities of the `bioimageio.spec` package, see the [bioimageio.spec package example notebook](https://github.com/bioimage-io/spec-bioimage-io/blob/main/example/load_model_and_create_your_own.ipynb), also available as a [Google Colab](https://colab.research.google.com/github/bioimage-io/spec-bioimage-io/blob/main/example/load_model_and_create_your_own.ipynb) notebook.

In [ ]:
from bioimageio.core import load_description

# Load the model description
# ------------------------------------------------------------------------------
if BMZ_MODEL_ID != "":
    model = load_description(BMZ_MODEL_ID)  
    print(f"\nThe model '{model.name}' with ID '{BMZ_MODEL_ID}' has been correctly loaded.")
elif BMZ_MODEL_DOI != "":
    model = load_description(BMZ_MODEL_DOI)  
    print(f"\nThe model '{model.name}' with DOI '{BMZ_MODEL_DOI}' has been correctly loaded.")
elif BMZ_MODEL_URL != "":
    model = load_description(BMZ_MODEL_URL)  
    print(f"\nThe model '{model.name}' with URL '{BMZ_MODEL_URL}' has been correctly loaded.")
else:
    print('\nPlease specify a model ID, DOI or URL')

if "draft" in BMZ_MODEL_ID or "draft" in BMZ_MODEL_DOI or "draft" in BMZ_MODEL_URL:
    print(f"\nThis is the DRAFT version of '{model.name}'. \nDraft versions have not been reviewed by the Bioimage Model Zoo Team and may contain harmful code. Run with caution.")

# To be added later:
# elif model.version != model.lastest_version:
#   print('\nThe loaded version of the model is: ' + model.version, 'but the latest version of the model is: ' + model.lastest_version)

# TODO: on the model loading success responses add version loaded



### 1.3 Inspect the model metadata

In [ ]:
print(f"The model '{model.name}' has the following properties and metadata\n")
print(f" Description:") 
pprint(model.description)

print("\n The authors of the model are: ")
pprint(model.authors)

print("\n It is maintained by:")
pprint(model.maintainers)

pprint("\n License: ")
pprint(model.license)

print("\n If you use this model, you are expected to cite it as: ")
pprint(model.cite)

print(f"\n Further documentation can be found here: {model.documentation.absolute()}")

if model.git_repo == None:
    print(f"\nNo associated GitHub repository.")
else:
    print(f"\n GitHub repository: ")
    pprint(model.git_repo)

print(f"\n Covers of the model '{model.name}' are: ")
for cover in model.covers:
    cover_data = imread(download(cover).path)
    plt.figure(figsize=(10, 10))
    plt.imshow(cover_data)
    plt.xticks([])
    plt.yticks([])
    plt.show()


### 1.4 Get expected model inputs and outputs

Each model expects Inputs and returns Outputs with specific features. The following code shows how to get the full metadata of the expected inputs and outputs of the model.

In [ ]:
print(f"Model '{model.name}' requires {len(model.inputs)} input(s) with the following features:")
for ipt in model.inputs:
    print(f"\ninput '{ipt.id}' with axes:")
    pprint(ipt.axes)
    print(f"Data description: {ipt.data}")
    print(f"Test tensor available at:  {ipt.test_tensor.source.absolute()}")
    if len(ipt.preprocessing) > 1:
        print("This input is preprocessed with: ")
        for p in ipt.preprocessing:
            print(p)

print("\n-------------------------------------------------------------------------------")
# # and what the model outputs are
print(f"Model '{model.name}' requires {len(model.outputs)} output(s) with the following features:")
for out in model.outputs:
    print(f"\noutput '{out.id}' with axes:")
    pprint(out.axes)
    print(f"Data description: {out.data}")
    print(f"Test tensor available at:  {out.test_tensor.source.absolute()}")
    if len(out.postprocessing) > 1:
        print("This output is postprocessed with: ")
        for p in out.postprocessing:
            print(p)

## 2. Test the model

The `bioimageio.core.test_model` function can be used to fully test the model.
This is done by running the predicition on the test input(s) and checking that they agree with the test output(s) provided in the model documentation.

This test should be run before using the model to ensure that it works properly.

`bioimageio.core.test_model` returns a dictionary with 'status'='passed'/'failed' and other detailed information.

In [ ]:
from bioimageio.core import test_model

test_summary = test_model(model)
test_summary.display()

## 3. Running a prediction

`bioimageio.core` implements functionality to run a prediction with models described in the `bioimage.io` format.

This includes functions to run predictions on `numpy.ndarray`s/`xarray.DataArrays` as input and convenience functions to run predictions for images stored on disc.

### 3.1. Load the test image and convert into a tensor

In [ ]:
from bioimageio.spec.utils import load_array
from bioimageio.spec.model import v0_5

assert isinstance(model, v0_5.ModelDescr)
input_image = load_array(model.inputs[0].test_tensor)
print(f"array shape: {input_image.shape}")

Create a `Tensor` (light wrapper around an `xarray.DataArray`) from the test input image. 

`bioimageio.core.Tensors/xarray.DataArrays` are like numpy arrays, but they have annotated axes.

The axes are used to validate that the axes of the input image match the axes expected by the model.

In [ ]:
from bioimageio.core import Tensor

test_input_tensor = Tensor.from_numpy(input_image, dims=model.inputs[0].axes)

# print the axis annotations ('dims') and the shape of the input array
print(f"tensor shape: {test_input_tensor.tagged_shape}")

A collection of tensors is called a `Sample`.

In the case of the `affable-shark` model it only has one input, but for models with multiple inputs a `Sample` includes a tensor for each input.

In [ ]:
from bioimageio.core import Sample

sample = Sample(members={"raw": test_input_tensor}, stat=None, id="sample-from-numpy")

sample

`bioimageio.core` provides a helper function `create_sample_for_model` to automatically create the `Sample` for the given model.

In [ ]:
from bioimageio.core.digest_spec import create_sample_for_model
from bioimageio.spec.utils import download

input_paths = {ipt.id: download(ipt.test_tensor).path for ipt in model.inputs}
print(f"input paths: {input_paths}")
assert isinstance(model, v0_5.ModelDescr)
sample = create_sample_for_model(
    model=model, inputs=input_paths, sample_id="my_demo_sample"
)

sample

There is also  a helper function `get_test_inputs` to directly import the test input sample for a given model.

In [ ]:
from bioimageio.core.digest_spec import get_test_inputs

test_sample = get_test_inputs(model)

test_sample

### 3.2. Create a prediciton pipeline

The `prediction_pipeline` function is used to run a prediction with a given model.

It applies the __pre-processing__, if indicated in the model rdf.yaml, runs __inference__ with the model and applies the __post-processing__, again if specified in the model rdf.yaml.

The `devices` argument can be used to specify which device(s), CPU, a single GPU, or multiple GPUs (not implemented yet), to use for inference with the model.

The default is `devices=None`, this will use a __GPU__ if available, otherwise it uses the __CPU__.


The `weight_format` argument can be used to specify which of the model's available weight formats to use.

The deafult is `weight_format=None`, this will use the weight format with highest priority (as defined by bioimageio.core).



In [ ]:
from bioimageio.core import create_prediction_pipeline

devices = None
weight_format = None

prediction_pipeline = create_prediction_pipeline(
    model, devices=devices, weight_format=weight_format
)

Use the new prediction pipeline to run a prediction for the previously loaded test image.

The prediction pipeline returns a `Sample` object, which will be displayed via napari.

In [ ]:
prediction: Sample = prediction_pipeline.predict_sample_without_blocking(sample)

# show the prediction result
show_images({**sample.members, **prediction.members})

### 3.3. Prediction without a PredicitionPipeline

`bioimageio.core` has two convenience functions `predict` and `predict_many` which allow the prediction of images without creating a `PredictionPipeline`.

In [ ]:
from bioimageio.core import predict  # , predict_many

# predict_many(model=model, inputs=[sample])

prediction: Sample = predict(model=model, inputs=sample)

# show the prediction result
show_images({**sample.members, **prediction.members})